In [ ]:
# Arnav - computeShadowPlanes()
# Output:
# shadow_planes -- length-T list of homogeneous 4-vectors, with the ith 
# list element storing the coordinates of the shadow plane (in the camera-centered 
# coordinate system) for the ith video frame

In [ ]:
# # IGNORE - MANUALLY UPLOADING VARS FOR TESTING - (SAVE FOR NOW)
# import numpy as np
# import cv2
# K = np.array([[1.94704892e+03, 0.00000000e+00, 5.71426625e+02],
#                 [0.00000000e+00, 1.82503130e+03, 5.36812397e+02],
#                 [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

# distCoeff = np.array([0.09107299, 0.1009922, -0.03594065, 0.00507835, -0.19710599])

# front_edge_times, image_lines = processVideo("./resources_unzipped/shadow2.MOV", mask_top, mask_bottom)
# # front_edge_times: tells frame T at which l_h or l_v hits for pixel (H,W)... [NOT NEEDED]
# # image_lines: for frame T, contains l_h and l_v (of shadow) on image plane

# Pi_h = np.array([ 0.03540478, -0.8460448,  -0.53193487, 19.20125639])
# Pi_v = np.array([-0.40653674,  0.45162227, -0.79420728, 32.20985317])


In [ ]:
def computeShadowPlanes(K, distCoeff, image_lines, Pi_h, Pi_v):  
    shadow_planes = []
    num_frames = len(image_lines)
    for T in range(num_frames):
        # choose corect frame
        l_v = image_lines[T][0]
        l_h = image_lines[T][1]

        # pick 2 points on l_h and l_v, inhom
        p1_h = np.array([0, -l_h[2]/l_h[1]])   # x-int
        p2_h = np.array([-l_h[2]/l_h[1], 0])   # y-int
        p1_v = np.array([0, -l_v[2]/l_v[1]])   # x-int
        p2_v = np.array([-l_v[2]/l_v[1], 0])   # y-int
        P = np.array([p1_h, p2_h, p1_v, p2_v]) 

        # backproj these points to get backproj rays
        P_rays = cv2.undistortPoints(P, K, distCoeff)
        P_rays = np.squeeze(P_rays)  # remove extraneous size-1 dimension 
        P_rays = np.concatenate([P_rays, np.ones((P_rays.shape[0], 1), dtype=np.float32)], axis=1)   # make hom
        P_rays_h = P_rays[0:2]  # only h rays
        P_rays_v = P_rays[2:4]  # only v rays

        # find intersect of _h with Pi_h and _v with Pi_v
        n_h = Pi_h[:-1]  # remove last elem, plane norm vec
        s_h = Pi_h[-1]   # only last elem, dist from origin
        llambda_h = -s_h/(np.dot(P_rays_h, n_h))  # create Nx1 array which has values of how much to scale every point
        threed_h = np.expand_dims(llambda_h, axis=1)*P_rays_h  # 2 3d points on Pi_h

        n_v = Pi_v[:-1]  # remove last elem, plane norm vec
        s_v = Pi_v[-1]   # only last elem, dist from origin
        llambda_v = -s_v/(np.dot(P_rays_v, n_v))  # create Nx1 array which has values of how much to scale every point
        threed_v = np.expand_dims(llambda_v, axis=1)*P_rays_v  # 2 3d points on Pi_v
        
        # turn Xi to homogeneous, and compute shadow plane with 4 points using SVD
        A = np.vstack((threed_h,threed_v))
        A = np.concatenate([A, np.ones((A.shape[0], 1), dtype=np.float32)], axis=1)  # make 3d points hom
        _,_,VT = np.linalg.svd(A)

        # plane of best-fit is the last row of V-transpose (i.e., last column of V)
        shadow_planes.append(VT[-1,])
    
    return np.array(shadow_planes)



In [ ]:
# # TESTING CELL... LEAVE FOR NOW
# shadow_planes = []
# num_frames = len(image_lines)
# for T in range(num_frames):
#     # choose corect frame
#     l_v = image_lines[T][0]
#     l_h = image_lines[T][1]

#     # pick 2 points on l_h and l_v, inhom
#     p1_h = np.array([0, -l_h[2]/l_h[1]])   # x-int
#     p2_h = np.array([-l_h[2]/l_h[1], 0])   # y-int
#     p1_v = np.array([0, -l_v[2]/l_v[1]])   # x-int
#     p2_v = np.array([-l_v[2]/l_v[1], 0])   # y-int
#     P = np.array([p1_h, p2_h, p1_v, p2_v]) 

#     # backproj these points to get backproj rays
#     P_rays = cv2.undistortPoints(P, K, distCoeff)
#     P_rays = np.squeeze(P_rays)  # remove extraneous size-1 dimension 
#     P_rays = np.concatenate([P_rays, np.ones((P_rays.shape[0], 1), dtype=np.float32)], axis=1)   # make hom
#     P_rays_h = P_rays[0:2]  # only h rays
#     P_rays_v = P_rays[2:4]  # only v rays

#     # find intersect of _h with Pi_h and _v with Pi_v
#     n_h = Pi_h[:-1]  # remove last elem, plane norm vec
#     s_h = Pi_h[-1]   # only last elem, dist from origin
#     llambda_h = -s_h/(np.dot(P_rays_h, n_h))  # create Nx1 array which has values of how much to scale every point
#     threed_h = np.expand_dims(llambda_h, axis=1)*P_rays_h  # 2 3d points on Pi_h

#     n_v = Pi_v[:-1]  # remove last elem, plane norm vec
#     s_v = Pi_v[-1]   # only last elem, dist from origin
#     llambda_v = -s_v/(np.dot(P_rays_v, n_v))  # create Nx1 array which has values of how much to scale every point
#     threed_v = np.expand_dims(llambda_v, axis=1)*P_rays_v  # 2 3d points on Pi_v
    
#     # turn Xi to homogeneous, and compute shadow plane with 4 points using SVD
#     A = np.vstack((threed_h,threed_v))
#     A = np.concatenate([A, np.ones((A.shape[0], 1), dtype=np.float32)], axis=1)  # make 3d points hom
#     _,_,VT = np.linalg.svd(A)

#     # plane of best-fit is the last row of V-transpose (i.e., last column of V)
#     shadow_planes.append(VT[-1,])



In [ ]:
# ALTERNATIVE METHOD... NOT NEEDED AND INCOMPLETE
# l_h_points_1 = []
# l_h_points_2 = []
# l_v_points_2 = []
# l_v_points_2 = []

# num_frames = image_lines.shape()[0]
# for T in range(num_frames):
#     # choose corect frame
#     l_h = image_lines[T, 0]
#     l_v = image_lines[T, 1]
#     l_h_points_1.append(np.array([0, -l_h[2]/l_h[1]]))
#     l_h_points_2.append(np.array([-l_h[2]/l_h[1], 0]))
#     l_v_points_1.append(np.array([0, -l_v[2]/l_v[1]]))
#     l_v_points_2.append([-l_v[2]/l_v[1], 0])

# def 2d_3d(input_array, plane):
#     input_rays = cv2.undistortPoints(P, K, distCoeff)
# # backproj these points to get backproj rays
# P_rays = cv2.undistortPoints(P, K, distCoeff)
# P_rays = np.squeeze(P_rays)  # remove extraneous size-1 dimension 
# P_rays = np.concatenate([P_rays, np.ones((P_rays.shape[0], 1), dtype=np.float32)], axis=1)   # make hom
# P_rays_h = P_rays[0:1]  # only h rays
# P_rays_v = P_rays[:-2]  # only v rays



# if we have 4xT input of all A's for each frame, can run SVD once to get all shadow planes
# [A1|A2|A3] [S1|S2|S3]* = 0 and has len T!
